In [130]:
import pathlib
import os
from fastai.vision.all import ImageDataLoaders, Resize, imagenet_stats, vision_learner, resnet34, error_rate, ClassificationInterpretation
from fastai.vision.data import get_image_files
import numpy as np

from fastai.vision.all import RandomResizedCrop, Flip, Dihedral, ToTensor, Normalize

# Ensure the path is correct and points to the directory containing images
chrono_images_path = pathlib.Path('/home/ubuntu/chrono-analyzer/data/')

# Get all image files from the directory
fnames = get_image_files(chrono_images_path)






In [ ]:
resize_transform = Resize(224)

# Define augmentation transforms
tfms = [
    Flip(p=0.5),  # Random horizontal flipping with 50% probability
    Dihedral() # Apply a Dihedral matrix to image
]


# Combine with normalization
final_tfms = tfms + [ToTensor(), Normalize.from_stats(*imagenet_stats)]

In [ ]:
# Extract filenames without paths (assuming filenames don't contain slashes)
# fnames = [os.path.basename(path) for path in fnames]
# fnames = [pathlib.Path(path) for path in fnames]

# Adjusted regular expression to capture only the label part
label_pattern = r"(.+)-.+-.+.jpg"


# Create DataLoaders using the extracted information
data = ImageDataLoaders.from_name_re(
    path=chrono_images_path,
    fnames=fnames,
    pat=label_pattern,
    item_tfms=resize_transform,
    batch_tfms = final_tfms,
    valid_pct=0.2,
)


In [ ]:
data.show_batch(figsize=(7,7))

In [ ]:
learner = vision_learner(data, resnet34, metrics=error_rate)


In [ ]:
learner.fit_one_cycle(5)